In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

%matplotlib inline

In [2]:
data = loadmat('ex3data1.mat')

data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

In [4]:
def sigmoid(z):  
    return 1 / (1 + np.exp(-z))

def cost(theta, X, y, learningRate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg

In [5]:
def gradient(theta, X, y, learningRate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y

    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)

    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)

    return np.array(grad).ravel()

In [6]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate):  
    rows = X.shape[0]
    params = X.shape[1]

    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))

    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))

        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x

    return all_theta

In [23]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[ -3.70247932e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -2.24803601e-10,   2.31962907e-11,   0.00000000e+00],
       [ -8.96250748e-05,   0.00000000e+00,   0.00000000e+00, ...,
          7.26120858e-09,  -6.19965325e-10,   0.00000000e+00],
       [ -8.39553375e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -7.61695635e-10,   4.64917650e-11,   0.00000000e+00],
       ..., 
       [ -7.00832718e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -6.92009371e-10,   4.29241678e-11,   0.00000000e+00],
       [ -7.65188092e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -8.09503517e-10,   5.31058853e-11,   0.00000000e+00],
       [ -6.63412494e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -3.49765987e-09,   1.13668573e-10,   0.00000000e+00]])

In [47]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    h = sigmoid(X*all_theta.T)
    
    h_argmax = np.argmax(h, axis=1)
    
    h_argmax = h_argmax + 1  
    
    return h_argmax

In [48]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a==b else 0 for (a,b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print('accuracy = {0}%'.format(accuracy*100))

accuracy = 74.6%
